# Optimization

## Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd

from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.core.problem import ElementwiseProblem

import src.optimization_utils as ou
import src.solutions as s


## Load Model

In [2]:
bundle = ou.load_model_bundle("../artifacts/mech_fail_model.joblib")

In [3]:
FEATURES = bundle["features"]
print(f"Training features ({len(FEATURES)}): {FEATURES}")


Training features (24): ['age', 'sex', 'PI_preop', 'PT_preop', 'LL_preop', 'SS_preop', 'T4PA_preop', 'L1PA_preop', 'SVA_preop', 'cobb_main_curve_preop', 'FC_preop', 'tscore_femneck_preop', 'HU_UIV_preop', 'HU_UIVplus1_preop', 'HU_UIVplus2_preop', 'num_levels', 'UIV_implant', 'num_fused_levels', 'ALIF', 'XLIF', 'TLIF', 'num_rods', 'num_screws', 'osteotomy']


In [4]:
UIV_CHOICES, xl, xu = ou.get_decision_config()

In [5]:
print("UIV_CHOICES:", UIV_CHOICES)
print("xl:", xl)
print("xu:", xu)

cols = ["uiv_code","num_fused_levels","ALIF","XLIF","TLIF","num_rods","num_screws","osteotomy"]
print(pd.DataFrame([xl, xu], index=["xl","xu"], columns=cols))

UIV_CHOICES: ['hook', 'PS', 'FS']
xl: [0 1 1 0 0 2 1 0]
xu: [2 4 1 1 1 4 4 1]
    uiv_code  num_fused_levels  ALIF  XLIF  TLIF  num_rods  num_screws  \
xl         0                 1     1     0     0         2           1   
xu         2                 4     1     1     1         4           4   

    osteotomy  
xl          0  
xu          1  


## Test Patient w fixed parameters

In [6]:
patient_fixed = {
    "age": 65,
    "sex": "FEMALE",
    "bmi": 18.48,
    "C7CSVL_preop": -6.1,
    "SVA_preop": 40.7,
    "T4PA_preop": 7.4,
    "L1PA_preop": 9.1,
    "LL_preop": 51.8,
    "L4S1_preop": 32.5,
    "PT_preop": 14.1,
    "PI_preop": 45.8,
    "SS_preop": 31.7,
    "cobb_main_curve_preop": 60.2,
    "FC_preop": 10.6,
    "tscore_femneck_preop": -0.5,
    "HU_UIV_preop": 229,
    "HU_UIVplus1_preop": 245,
    "HU_UIVplus2_preop": 248,
}

ou.debug_candidate(
    x=np.array([0, 10, 1, 0, 0, 2, 20, 0]),
    patient_fixed=patient_fixed,
    bundle=bundle,
    uiv_choices=UIV_CHOICES,
)

{'x': [0, 10, 1, 0, 0, 2, 20, 0],
 'plan': {'UIV_implant': 'hook',
  'num_fused_levels': 10,
  'ALIF': 1,
  'XLIF': 0,
  'TLIF': 0,
  'num_rods': 2,
  'num_screws': 20,
  'osteotomy': 0},
 'mech_fail_prob': 0.35360197332460813,
 'fitness': 0.35360197332460813}

## Build optimization problem

In [7]:
def make_problem():
    def _evaluate(self, x, out, *args, **kwargs):
        f = ou.fitness_mech_fail_only(x, patient_fixed, bundle, UIV_CHOICES)
        out["F"] = np.array([f], dtype=float)

    ProblemType = type(
        "SpineProblem",
        (ElementwiseProblem,),
        {
            "__init__": lambda self: ElementwiseProblem.__init__(
                self,
                n_var=len(xl),
                n_obj=1,
                xl=xl,
                xu=xu,
                vtype=int,
            ),
            "_evaluate": _evaluate,
        },
    )
    return ProblemType()

problem = make_problem()

## Run GA and view results

In [8]:
algorithm = GA(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=3),
    eliminate_duplicates=True,
)

res = minimize(
    problem,
    algorithm,
    get_termination("n_gen", 15),
    seed=42,
    verbose=True,
    save_history=True
)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       94 |  0.5461440432 |  0.0475638321
     2 |      194 |  0.0856727984 |  0.0456730572
     3 |      294 |  0.0608630762 |  0.0442672165
     4 |      394 |  0.0521026573 |  0.0442672165
     5 |      494 |  0.0486717719 |  0.0442672165
     6 |      594 |  0.0465194888 |  0.0438539842
     7 |      694 |  0.0450650575 |  0.0421041650
     8 |      794 |  0.0442249244 |  0.0421041650
     9 |      894 |  0.0442125274 |  0.0421041650
    10 |      994 |  0.0441444725 |  0.0421041650
    11 |     1094 |  0.0440124948 |  0.0421041650
    12 |     1194 |  0.0436928816 |  0.0421041650
    13 |     1294 |  0.0431015619 |  0.0421041650
    14 |     1394 |  0.0422791469 |  0.0421041650
    15 |     1494 |  0.0421041650 |  0.0421041650


## Show best solution

In [9]:
best_x = np.asarray(res.X).astype(int)
best_plan = ou.decode_plan(best_x, UIV_CHOICES)

best_full = ou.build_full_input(patient_fixed, best_plan)
best_prob = ou.predict_mech_fail_prob(best_full, bundle)

print("Best mech_fail_prob:", best_prob)
best_plan

Best mech_fail_prob: 0.042104165015110856


{'UIV_implant': 'PS',
 'num_fused_levels': 1,
 'ALIF': 1,
 'XLIF': 1,
 'TLIF': 0,
 'num_rods': 4,
 'num_screws': 1,
 'osteotomy': 0}

## Getting multiple solutions

Currently forcing ALIF to 1 for all solutions because that is all we can evaluate with training data

**Get top 10 solutions from last generation**

In [10]:
pop = res.algorithm.pop
X_pop = pop.get("X")
F_pop = pop.get("F").flatten()

order = np.argsort(F_pop)

TOP_SLICE = 1500
TOP_N = 10

rows = []
for idx in order[:TOP_SLICE]:
    x = np.asarray(X_pop[idx]).astype(int)
    plan = ou.decode_plan(x, UIV_CHOICES)
    rows.append({
        **plan,
        "fitness": float(F_pop[idx]),
    })

df = pd.DataFrame(rows)

df_unique = df.drop_duplicates(
    subset=["UIV_implant", "num_fused_levels", "ALIF", "XLIF", "TLIF",
            "num_rods", "num_screws", "osteotomy"],
    keep="first",
).reset_index(drop=True)

df_unique.head(TOP_N)


,UIV_implant,num_fused_levels,ALIF,XLIF,TLIF,num_rods,num_screws,osteotomy,fitness
0,PS,1,1,1,0,4,1,0,0.042104


In [11]:
df["UIV_implant"].value_counts()


UIV_implant
PS    100
Name: count, dtype: int64

In [12]:
solutions = s.get_diverse_solutions(
    res=res,
    top_n=5,
    top_per_gen=50,
    eps=0.1,
    bucket_cols=("UIV_implant", "XLIF", "TLIF"),
    n_per_bucket=1,
)

solutions


,UIV_implant,num_fused_levels,ALIF,XLIF,TLIF,num_rods,num_screws,osteotomy,fitness,gen
0,PS,1,1,1,0,4,1,0,0.042104,6
1,PS,1,1,0,0,4,1,0,0.044267,2
2,hook,1,1,0,0,4,1,0,0.071575,0
3,FS,1,1,0,0,4,1,0,0.073552,0
4,hook,1,1,1,0,4,3,0,0.073781,1
